# GCP Dataflow

O Dataflow é um serviço de processamento de dados em lote e em tempo real, que permite criar pipelines de dados escaláveis e eficientes. Ele é baseado no modelo de programação Apache Beam, que permite escrever código uma vez e executá-lo em diferentes ambientes de execução, como o Dataflow, Spark e Flink.

Para maiores informações, consulte a [documentação oficial do Dataflow](https://cloud.google.com/dataflow/docs).

## 1. Inicialização do ambiente Dataproc

O primeiro passo é iniciar um cluster do Google Cloud Dataproc, que será utilizado para executar os notebooks.

### 1.1. Criando um cluster Dataproc com o Cloud Shell

Existem várias formas de iniciar um cluster do Dataproc, uma das formas mais simples, é utilizar o comando `gcloud` no terminal.

```bash
gcloud dataproc clusters create $USER \
    --region us-central1 \
    --zone us-central1-a \
    --subnet=default \
    --delete-max-idle=120m \
    --single-node \
    --master-machine-type n2-standard-4 \
    --master-boot-disk-size 50GB \
    --image-version 2.2-debian12 \
    --enable-component-gateway \
    --optional-components=JUPYTER,ICEBERG \
    --project pdm-savio
```

Caso prefira, você pode rodar esse comando no Google Cloud Shell, para acessá-lo, clique no ícone do terminal no canto superior direito do console do GCP.
Após clicar no ícone, uma janela irá se abrir na parte inferior da tela, onde você poderá executar o comando acima para criar o cluster do Dataproc.
Abaixo tem uma imagem ilustrando onde clicar para abrir o Cloud Shell.

![Cloud Shell](imagens-dataflow/img2-cloud-shell.png)

## 2. Copiando o notebook para o Dataproc

Esse notebook está disponível no repositório do GitHub: [robertogyn19/aula-pdm-pubsub](https://github.com/robertogyn19/aula-pdm-pubsub).
Para copiar o notebook para o Dataproc, você pode fazer o clone do projeto para dentro do cluster, utilizando o comando abaixo:
```bash
git clone https://github.com/robertogyn19/aula-pdm-pubsub.git
```

Agora sim, você também tem acesso a todas as instruções para essa aula.

## 3. Apache Beam - Exemplo básico

Para entender como funciona o Dataflow, vamos criar um exemplo básico utilizando o Apache Beam.
Instale o pacote `apache-beam`:
```bash
pip install apache-beam[gcp]
```

O pacote `apache-beam[gcp]` inclui as dependências necessárias para executar pipelines no Google Cloud Dataflow.

In [2]:
%pip install apache-beam[gcp]

Note: you may need to restart the kernel to use updated packages.


Vamos executar o código clássico de wordcount.
```bash
$ python -m apache_beam.examples.wordcount --output wordcount-local-output
INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.io.iobase:*** WriteImpl min_shards undef so its 1, and we write per Bundle
INFO:apache_beam.runners.worker.statecache:Creating state cache with size 104857600
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.01 seconds.
```

O comando acima irá executar o exemplo de wordcount localmente, salvando a saída no arquivo `wordcount-local-output-00000-of-00001`.
Você pode executar o comando abaixo para obter a lista das 10 palavras mais frequentes:
```bash
$ sort -t: -k2 -nr wordcount-local-output-00000-of-00001 | head -n 10
the: 786
I: 622
and: 594
of: 447
to: 438
my: 402
you: 401
a: 366
in: 271
not: 266
```

### 3.1. Explicando o código (1)

O código do exemplo de wordcount está disponível no repositório do GitHub do Apache Beam, [wordcount.py](https://github.com/apache/beam/blob/master/sdks/python/apache_beam/examples/wordcount.py).

A função `WordExtractingDoFn` é responsável por extrair as palavras de cada linha do arquivo de entrada.

```python
class WordExtractingDoFn(beam.DoFn):
  """Parse each line of input text into words."""
  def process(self, element):
    """Returns an iterator over the words of this element.

    The element is a line of text.  If the line is blank, note that, too.

    Args:
      element: the element being processed

    Returns:
      The processed element.
    """
    return re.findall(r'[\w\']+', element, re.UNICODE)
```

In [4]:
import re

# Para ver na prática o que a função findall faz, vamos rodar o comando abaixo.
linha = "Aula de PDM sobre Dataflow"
re.findall(r'[\w\']+', linha, re.UNICODE)

['Aula', 'de', 'PDM', 'sobre', 'Dataflow']

### 3.2. Explicando o código (2)

O trecho abaixo foi recortado do exemplo, vamos acompanhar os comentários para entender o que está acontecendo.

```python
# Inicialização do pipeline com os argumentos passados, ex.: --output
pipeline = beam.Pipeline(options=pipeline_options)

# Primeira etapa do pipeline, a entrada de dados, o valor em known_args.input é o arquivo de entrada
# Como não passamos um valor, será utilizado um padrão (gs://dataflow-samples/shakespeare/kinglear.txt)
lines = pipeline | 'Read' >> ReadFromText(known_args.input)

# Segunda etapa do pipeline, a contagem das palavras, isso é feito em 3 etapas:
# +-------------------------+     +--------------------------+     +--------------------+
# | 1) Divisão das palavras | --> | 2) Atribuição de valores | --> | 3) Agrupamento e   |
# |   ("tokenização")       |     |   (ex.: palavra -> 1)    |     |    soma (reduce)   |
# +-------------------------+     +--------------------------+     +--------------------+
#
# 1. Transformação das linhas em palavras, a função WordExtractingDoFn que faz isso
# 
# 2. Atribuição do valor 1 para cada palavra, no nosso exemplo acima, o resultado seria assim:
#   Aula    : 1
#   de      : 1
#   PDM     : 1
#   sobre   : 1
#   Dataflow: 1
# 
# 3. Agrupa as palavras iguais e soma os valores atribuídos no passo anterior.
# 
counts = (
  lines
  | 'Split' >> (beam.ParDo(WordExtractingDoFn()).with_output_types(str))
  | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
  | 'GroupAndSum' >> beam.CombinePerKey(sum))


# Format the counts into a PCollection of strings.
def format_result(word, count):
  return '%s: %d' % (word, count)

output = counts | 'Format' >> beam.MapTuple(format_result)

# Última etapa do pipeline, a escrita do resultado no arquivo que passamos como parâmetro
output | 'Write' >> WriteToText(known_args.output)

# Execução do pipeline
result = pipeline.run()
result.wait_until_finish()
```

## 4. Dataflow - Runners: DirectRunner e DataflowRunner

O exemplo de wordcount é um pipeline simples que lê um arquivo de texto, conta as palavras e escreve a saída em um arquivo.

O comando acima executou o código com o `DirectRunner`, que é o executor local do Apache Beam.

Antes de executar o pipeline no Google Cloud Dataflow, é necessário configurar o ambiente e associar ao projeto no Google Cloud Platform (GCP).

Primeiramente, precisamos configurar algumas variáveis de ambiente:

```bash
export PROJECT_ID=pdm-savio
export PROJECT_NUMBER=559515741601
export USER_IDENTIFIER=robertogyn19@gmail.com
```

As informações de `PROJECT_ID` e `PROJECT_NUMBER` podem ser obtidas no console do GCP, na seção "IAM e administrador" > "Configurações" ([link](https://console.cloud.google.com/iam-admin/settings)).

![Configurações do projeto](imagens-dataflow/img1-settings.png)

Com as variáveis de ambiente configuradas, vamos configurar as permissões necessárias para o usuário executar o pipeline no Dataflow.
```bash
gcloud projects add-iam-policy-binding $PROJECT_ID --member="user:$USER_IDENTIFIER" --role=roles/iam.serviceAccountUser

gcloud projects add-iam-policy-binding $PROJECT_ID \
--member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
--role=roles/dataflow.admin

gcloud projects add-iam-policy-binding $PROJECT_ID \
--member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
--role=roles/dataflow.worker

gcloud projects add-iam-policy-binding $PROJECT_ID \
--member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
--role=roles/storage.objectAdmin

Agora vamos executar o mesmo código utilizando o `DataflowRunner`, que é o executor do Google Cloud Dataflow.

```bash
python -m apache_beam.examples.wordcount \
    --runner DataflowRunner \
    --project pdm-savio \
    --region us-central1 \
    --temp_location gs://tmp-pdm/dataflow/ \
    --output gs://tmp-pdm/wordcount-output-$USER
```

Agora vamos acessar a página do [Dataflow](https://console.cloud.google.com/dataflow/jobs/us-central1) para ver o nosso job.
![dataflow-job](imagens-dataflow/img3-wordcount-job.png)

### 4.1. Dataflow - Outros runners

Além dos runners Direct utilizado para testes e o GCP Dataflow, existem também:
1. Apache Flink Runner:
    * Usa o motor Apache Flink (muito forte em streaming de baixa latência).
    * Suporte robusto a stateful processing e event-time. 
2. Apache Spark Runner:
    * Usa o motor Apache Spark
    * Bom para workloads batch grandes (Spark SQL, MLlib, etc.).
3. Samza Runner:
   * Usa Apache Samza (mantido pelo LinkedIn).
   * Focado em streaming com integração a Kafka e YARN.